# Parsing Debug Notebook

This somewhat messy notebook makes it easier to debug the parser, because we can just rerun the cells needed to set up the debugging process with various parameters.

In [1]:
import json
import math
import re
from pathlib import Path

import boto3
import pandas as pd

from parse_990_textract.bucket import open_df
from parse_990_textract.filing import create_roadmap, extract_from_roadmap
from parse_990_textract.models import BoundingBox, TableExtractor
from parse_990_textract.parse import create_extractors, find_item, find_pages
from parse_990_textract.postprocessing import clean_filing, clean_f_i, clean_f_ii, clean_f_iii, postprocess
from parse_990_textract.setup import load_extractor_df
from parse_990_textract.table import extract_table_data, find_table_pages, create_tablemap
from parse_990_textract.utils import get_coordinate, get_regex, cluster_words

In [2]:
bucket = boto3.resource("s3").Bucket("s3-ocr-990s-demo")

In [3]:
validation_data = pd.read_csv("validation_data.csv", index_col="job_id").fillna("")
validation_data.head()

,pdf_key,other_expenses_c_mgmt_general,payments_affiliates_total,other_expenses_d_prog_service,total_revenue,fees_for_lobbying_services_total,other_employee_benefits_fundraising,total_fundraising_expenses,compensation_officers_prog_service,fees_for_mgmt_services_total,...,travel_total,pension_plan_acc_contrib_prog_service,advertising_promotion_mgmt_general,pension_plan_acc_contrib_mgmt_general,compensation_disq_persons_total,other_salaries_wages_mgmt_general,office_expenses_mgmt_general,fees_for_other_services_fundraising,total_number_other_recipient_foreign_orgs_entities,activities_per_region_totals_total_expenditure
job_id,,,,,,,,,,,,,,,,,,,,,
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4,EIN_760733035_YEAR_2009_FORMTYPE_990.pdf,,,33000.0,1415024,,,4503.0,,,...,18307.0,,,,,,,,,
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc,EIN_363235550_YEAR_2009_FORMTYPE_990.pdf,,,15832.0,9899921,,27161.0,597789.0,295722.0,,...,677722.0,34261.0,30,3462.0,,159156.0,48381.0,22350.0,,
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b,EIN_223134995_YEAR_2010_FORMTYPE_990.pdf,0.0,,10823.0,3370595,,12776.0,263765.0,148689.0,2250.0,...,197092.0,,127381,,396855.0,103312.0,11723.0,23021.0,,
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82,EIN_264320885_YEAR_2009_FORMTYPE_990.pdf,,,,46117,,,669.0,,,...,,,,,,,458.0,,,
1dd5dc37f4e99e27668c769d04d0f72273a6c582d504bf0b88d35d5a4c15c1be,EIN_581943161_YEAR_2009_FORMTYPE_990.pdf,581.0,,0.0,583209,,2521.0,14409.0,298484.0,,...,7233.0,,,,,,4010.0,0.0,,


In [4]:
extractor_df = load_extractor_df("990_extractors.csv")
roadmap_df = load_extractor_df("990_roadmap.csv")
schedule_f_tablemap_df = load_extractor_df("schedule_f_table_roadmap.csv")
schedule_f_table_extractor_df = pd.read_csv("schedule_f_table_extractors.csv")
schedule_f_row_extractor_df = pd.read_csv("schedule_f_row_extractors.csv")

In [5]:
PART_I_HEADER = r"\(a\)\s*Region|\(d\)\s*Activities|\(e\)\s*If activity|\(f\)Total expenditures"
PART_II_HEADER = r"\(b\)\s*IRS code|\(c\)\s*Region|\(d\)\s*Purpose|\(f\)\s*Manner|\(h\)\s*Description"
PART_III_HEADER = r"\(b\)\s*Region|\(e\)\s*Manner of cash|\(h\)\s*Method of va"
PART_I_TABLE_NAME = "Activities per Region"
PART_II_TABLE_NAME = r"Grants to Organizations Outside the United States"
PART_III_TABLE_NAME = "Grants to Individuals Outside the United States"

In [6]:
filing_rows = []
schedule_f_part_i_rows = []
schedule_f_part_ii_rows = []
schedule_f_part_iii_rows = []


for i, job_id in enumerate(validation_data.index.values):
    print(i)
    print(job_id)
    pdf_key = validation_data.at[job_id, "pdf_key"]
    print(pdf_key)
    
    data = open_df(bucket, job_id)
    lines = data.loc[data["BlockType"] == "LINE"]
    words = data.loc[data["BlockType"] == "WORD"]
    page_map = find_pages(lines)
    roadmap = create_roadmap(
        lines, roadmap_df, page_map
    )
    
    row = extract_from_roadmap(
        words, lines, roadmap, extractor_df, page_map
    )
    row = postprocess(row, job_id, pdf_key, clean_filing)
    filing_rows.append(row)
    
    pages = lines.groupby("Page")
    
    part_i_table = extract_table_data(
        pages, lines, words, PART_I_HEADER, PART_I_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_i_table = postprocess(part_i_table, job_id, pdf_key, clean_f_i)
    if part_i_table is not None:
        schedule_f_part_i_rows.append(
            part_i_table
        )
    part_ii_table = extract_table_data(
        pages, lines, words, PART_II_HEADER, PART_II_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_ii_table = postprocess(part_ii_table, job_id, pdf_key, clean_f_ii)
    if part_ii_table is not None:
        schedule_f_part_ii_rows.append(
            part_ii_table
        )
    part_iii_table = extract_table_data(
        pages, lines, words, PART_III_HEADER, PART_III_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_iii_table = postprocess(part_iii_table, job_id, pdf_key, clean_f_iii)
    if part_iii_table is not None:
        schedule_f_part_iii_rows.append(
            part_iii_table
        )

0
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4
EIN_760733035_YEAR_2009_FORMTYPE_990.pdf


No match for year_formation in L Year of Formation M State of legal domicile
No match for state_of_domicile in M State of legal domicile


1
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc
EIN_363235550_YEAR_2009_FORMTYPE_990.pdf


No match for total_unrelated_biz_revenue in 7a NONE
No match for net_unrelated_biz_taxable_revenue in 7b NONE
No match for benefits_paid_members_expenses in NONE
No match for professional_fundraising_fees_expenses in NONE
No match for grants_foreign_individuals_govt_orgs_total in NONE
No match for benefits_to_members_total in NONE
No match for compensation_disq_persons_total in NONE
No match for fees_for_mgmt_services_total in NONE
No match for fees_for_lobbying_services_total in NONE
No match for fees_for_fundraising_services_total in NONE
No match for fees_for_investment_mgmt_services_total in NONE
No match for royalties_total in NONE
No match for entertainment_travel_govt_officials_total in NONE
No match for interest_total in NONE
No match for payments_affiliates_total in NONE


2
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b
EIN_223134995_YEAR_2010_FORMTYPE_990.pdf
3
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82
EIN_264320885_YEAR_2009_FORMTYPE_990.pdf
4
1dd5dc37f4e99e27668c769d04d0f72273a6c582d504bf0b88d35d5a4c15c1be
EIN_581943161_YEAR_2009_FORMTYPE_990.pdf
5
5596f51a999ebbd4cb992f490ceaffcddbac9bce532b911997683ae6897c2797
EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
6
481a62c75776cc7f62075c1d60c230ea01f2dbc5b6224c65cb73db1728e86b00
EIN_954806856_YEAR_2009_FORMTYPE_990.pdf
7
39d270117e4b6354850551c8237421403da7d2b3f5d06657a28f5a9d07febf17
EIN_521238301_YEAR_2010_FORMTYPE_990.pdf


...

<class 'KeyError'>: '(a) Name of organization'


22
a6529e504df346097da99104a353e977426e018cd5ac33b62cd2dd89c90763c5
EIN_311002913_YEAR_2008_FORMTYPE_990.pdf
23
d1925c2d74adaa3f150ded3ce67dfe7ae3a306f0db0289ad5755a28d801b2b0b
EIN_202408857_YEAR_2010_FORMTYPE_990.pdf


No match for activities_per_region_totals_number_of_offices in Act the


24
01d89ee5d14575c1321b2e4d67431d172ba76212b4a266bdaf474275029fd78b
EIN_521830327_YEAR_2009_FORMTYPE_990.pdf


No match for total_fundraising_expenses in b Total fundraising expenses (Part IX, column (D), line 25)
No match for revenue_less_expenses in <43,775.> End of Year


In [7]:
output_data = pd.concat(filing_rows).reset_index(drop=True).set_index("job_id")

In [8]:
output_data.head()

field_name,index,name,address,city,state,zip,website,gross_receipts,year_formation,state_of_domicile,...,total_functional_expense_fundraising,activities_per_region_totals_number_of_offices,activities_per_region_totals_number_of_employees,activities_per_region_totals_total_expenditure,total_number_recipient_foreign_orgs_listed_as_charities,total_number_other_recipient_foreign_orgs_entities,pdf_key,ein,year,filing_id
job_id,,,,,,,,,,,,,,,,,,,,,
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4,0,MAKE WAY PARTNERS INC,PO BOX 26367,BIRMINGHAM,OX,26367,www MAKEWAYPARTNERS ORG,1426217,,,...,4503,,,,,,EIN_760733035_YEAR_2009_FORMTYPE_990.pdf,760733035,2009,760733035_2009
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc,0,THE FEDERALIST SOCIETY FOR LAW AND,"1015 18TH ST., N.W. 425",WASHINGTON,DC,20036,www. FED-SOC.ORG,11033302,1982,IL,...,597789,,,,,,EIN_363235550_YEAR_2009_FORMTYPE_990.pdf,363235550,2009,363235550_2009
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b,0,Ron Hutchcraft Ministries Inc,PO Box 400,Harnson,AR,20400,www hutchcraft com,3390,1991,NJ,...,263765,,,,,,EIN_223134995_YEAR_2010_FORMTYPE_990.pdf,223134995,2010,223134995_2010
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82,0,FREEDOM 4 24,21430 TIMBERLAKE ROAD STE 101,LYNCHBURG,VA,24502,freedom424 org,61857,2009,VA,...,669,,,,,,EIN_264320885_YEAR_2009_FORMTYPE_990.pdf,264320885,2009,264320885_2009
1dd5dc37f4e99e27668c769d04d0f72273a6c582d504bf0b88d35d5a4c15c1be,0,GEORGIA PUBLIC POLICY FOUNDTION,6100 LAKE FORREST DR,LAKE FORREST,GA,30328,www GPPF org,583209,1991,GA,...,14409,,,,,,EIN_581943161_YEAR_2009_FORMTYPE_990.pdf,581943161,2009,581943161_2009


In [9]:
def clean(x):
    x = str(x)
    x = re.sub(r"\.0\b", "", x)
    x = re.sub("\D", "", x)
    return x

In [10]:
def compare_output(to_validate, to_compare, col):
    return pd.DataFrame(
        {
            "extracted": to_validate.loc[col].loc[
                lambda series: series != to_compare.loc[col]
            ],
            "expected": to_compare.loc[col].loc[
                lambda series: series != to_validate.loc[col]
            ],
        }
    )

In [11]:
to_compare = validation_data.set_index("pdf_key").applymap(clean)
to_validate = output_data[validation_data.columns].set_index("pdf_key").applymap(clean)

for col in to_validate.index:
    validated = compare_output(to_validate, to_compare, col)
    if validated.any().any():
        print(col)
        print(f"{validated.shape[0]} mismatched items.")
        print(validated)
        print("-"*79)

EIN_113489123_YEAR_2008_FORMTYPE_990.pdf
2 mismatched items.
              extracted expected
total_revenue      1017  1017506
travel_total        137    13710
-------------------------------------------------------------------------------


In [12]:
assert False

AssertionError: 

## BREAK

In [13]:
pd.concat(schedule_f_part_i_rows).head(50)

field,index,region,number_offices,number_employees,activities_conducted,specific_type_activity,total_expenditures,job_id,pdf_key,ein,year,filing_id
0,0,,,,the region),service(s) region,,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf,231352689,2009,231352689_2009
1,1,Europe,1,,training,offers a Th M degree,6661,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf,231352689,2009,231352689_2009
2,2,Totals,1,0,,,6661,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf,231352689,2009,231352689_2009
0,0,(1),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
1,1,(2),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
2,2,(3),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
3,3,(4),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
4,4,(5),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
5,5,(6),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
6,6,(7),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010


In [14]:
pd.concat(schedule_f_part_i_rows).iloc[50:100]

field,index,region,number_offices,number_employees,activities_conducted,specific_type_activity,total_expenditures,job_id,pdf_key,ein,year,filing_id
16,16,East Asia and Pacific,,,Programservices,Student services,22043,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
17,17,East Asia and Pacific,,,Investments,,,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
18,18,Europe Including Iceland & Greenland),,,Fundraising,,59246,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
19,19,Iceland,,,Grantmaking,,5351568,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
20,20,Europe Including Iceland freenland,,,Program Services,Service Centers,245139,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
21,21,Europe Including Iceland 8 Greenland,,,Progr Services,Academic Support,689566,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
22,22,cluding Iceland Greenland),,,Progr Services,institutional Support,80288,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
23,23,Europe Including Iceland ;reenland),,,Program Services,Instruction,4676915,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
24,24,urope including Icelan Greenland),,15,Progr Services,Research other academic activity,8736896,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
25,25,urope cluding Iceland Greenland),,,Prog Services,Student Services,321429,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010


In [15]:
pd.concat(schedule_f_part_ii_rows).iloc[:50]

field,index,org_name,irs_code,region,grant_purpose,amount_cash,manner_cash,amount_noncash,desc_noncash,method_valuation,job_id,pdf_key,ein,year,filing_id
0,0,2 Enter total number has provided,of organizations a section 501(c)(3),that are recognized equivalency letter,as charities,,or for which the,,,,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf,231352689,2009,231352689_2009
1,1,3 Enter total number,of other organizations,or entities,,,,,,,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf,231352689,2009,231352689_2009
2,2,,applicable),,,,,,,"appraisal, other)",5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf,231352689,2009,231352689_2009
0,0,,,,,,,,,(1),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
1,1,,,,,,,,,(2),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
2,2,,,,,,,,,(3),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
3,3,,,,,,,,,(4),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
4,4,,,,,,,,,(5),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
5,5,,,,,,,,,(6),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
6,6,,,,,,,,,(7),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010


In [ ]:
table_test_df = open_df(bucket, "67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebdb32d3186b19d6a6fd5")

In [ ]:
test_lines = table_test_df.loc[
    table_test_df["BlockType"] == "LINE"
]
test_words = table_test_df.loc[
    table_test_df["BlockType"] == "WORD"
]
test_pages = test_lines.groupby("Page")

In [ ]:
part_i_table_pages = find_table_pages(
    test_pages["Text"].agg(lambda words: " ".join(words)),
    PART_I_HEADER,
)

In [ ]:
part_i_table_pages

In [ ]:
part_i_tablemaps = pd.DataFrame(
    {
        "page": part_i_table_pages,
        "tablemap": part_i_table_pages.map(
            lambda page: create_tablemap(
                test_lines, schedule_f_tablemap_df, page
            ).dropna()
        ),
    }
)

In [ ]:
part_i_tablemaps["tablemap"].iloc[0]

In [ ]:
part_i_row_extractors = schedule_f_row_extractor_df.loc[
    schedule_f_row_extractor_df["table"] == PART_I_TABLE_NAME
]
part_i_table_data = schedule_f_table_extractor_df.loc[
    schedule_f_table_extractor_df["table"] == PART_I_TABLE_NAME
].iloc[0]

In [ ]:
rows = part_i_tablemaps.assign(
    extractor=part_i_tablemaps["tablemap"].map(
        lambda tablemap: TableExtractor(
            header_top_label=part_i_table_data["header_top"],
            top_label=part_i_table_data["table_top"],
            bottom_label=part_i_table_data["table_bottom"],
            tablemap=tablemap,
            fields=part_i_row_extractors["field"],
        )
    )
)

In [ ]:
test_extractor = rows["extractor"].iloc[1]

In [ ]:
test_extractor.get_col_spans(test_words, 30)

In [ ]:
test_extractor.extract_rows(test_words, 30).head(50)

In [ ]:
test_extractor.get_col_spans(test_words, 30)

In [ ]:
table_words = test_extractor.get_table_words(test_words, 30)

In [ ]:
table_words.loc[
    table_words["Text"].str.contains("44"),
    "Midpoint_X"
]

In [ ]:
word_clusters = cluster_words(table_words, table_words["Height"].min(), "Midpoint_Y")
[" ".join(word.sort_values(by="Left")["Text"].values) for word in word_clusters]

In [ ]:
def columnize(word_cluster, col_spans):
    return col_spans.map(
        lambda span: word_cluster.loc[
            word_cluster["Midpoint_X"].between(*span)
        ]
    )


def get_cluster_coords(cluster):
    cluster_coords = {
        "Left": cluster["Left"].min(),
        "Right": cluster["Right"].max(),
        "Height": cluster["Height"].max(),
        "Midpoint_X": cluster["Midpoint_X"].median(),
        "Midpoint_Y": cluster["Midpoint_Y"].median(),
        "Top": cluster["Top"].min(),
        "Bottom": cluster["Bottom"].min(),
    }
    cluster_coords["Width"] = cluster_coords["Right"] - cluster_coords["Left"]
    return cluster_coords

col_spans = test_extractor.get_col_spans(test_words, 30)

columnize(word_clusters[0], col_spans).map(get_cluster_coords)

In [ ]:
y_tol = table_words["Height"].max() * 1.5
x_tol = table_words["Width"].median()

columnized = columnize(word_clusters[0], col_spans)
last_col_coords = pd.DataFrame.from_records(
    columnized.map(
        get_cluster_coords
    )
)
sum_y_delta = last_col_coords["Height"].max()
print(sum_y_delta)
rows = []
current_row = [columnized]
print(" ".join(word_clusters[0].sort_values(by="Left")["Text"].values))
for count, cluster in enumerate(word_clusters[1:]):
    print("-"*50)
    print(" ".join(cluster.sort_values(by="Left")["Text"].values))
    columnized = columnize(cluster, col_spans)
    col_coords = pd.DataFrame.from_records(columnized.map(get_cluster_coords))
    nonempty = col_coords.dropna().index.to_series()
    last_nonempty = last_col_coords.dropna().index.to_series()
    delta_cols = (
        (nonempty.count() > last_nonempty.count())
        or (~nonempty.isin(last_nonempty)).any()
    )
    y_delta = (
        col_coords["Midpoint_Y"].median()
        - last_col_coords["Midpoint_Y"].median()
    )
    sum_y_delta += y_delta
    mean_y_delta = sum_y_delta / (count + 1)
    min_y_delta = mean_y_delta*0.75
    print("Y Delta:", y_delta)
    print("Mean Y Delta:", mean_y_delta)
    print("More cols detected", delta_cols)
    print("Y tolerance exceeded", (y_delta > y_tol))
    if (
        (
            delta_cols 
            or (y_delta > y_tol)
        )
        and (y_delta > min_y_delta)
    ):
        combined_row = pd.Series([
            line.map(
                lambda x: x.sort_values(
                    by="Left"
                ).reset_index(drop=True)["Text"].fillna("")
            ).agg(
                lambda x: " ".join(x.values)
            ) + " "
            for line in current_row
        ]).sum().str.strip()
        print(combined_row)
        rows.append(combined_row)
        current_row = [columnized]
    else:
        current_row.append(columnized)
    last_col_coords = col_coords
print("-"*50)
combined_row = pd.Series([
    line.map(
        lambda x: x.sort_values(
            by="Left"
        ).reset_index(drop=True)["Text"].fillna("")
    ).agg(
        lambda x: " ".join(x.values)
    ) + " "
    for line in current_row
]).sum().str.strip()
print(combined_row)
rows.append(combined_row)

In [ ]:
nonempty.reset_index(drop=True) == last_nonempty.reset_index(drop=True)

In [ ]:
header_words = test_extractor.get_header_words(test_words, 30)

In [ ]:
header_words.sort_values(by="Left")[["Text", "Left", "Right", "Midpoint_X", "Width"]]

In [ ]:
def cluster_x(words, tolerance):
    if (tolerance == 0) or (words.shape[0] < 2):
        return [
            [word] for (idx, word)
            in words.sort_values(by="Left").iterrows()
        ]
    groups = []
    sorted_words = words.sort_values(by="Left")
    current_group = [sorted_words.iloc[0]]
    last = sorted_words.iloc[0]["Right"]
    for idx, word in sorted_words.iloc[1:].iterrows():
        if word["Left"] <= (last + tolerance):
            current_group.append(word)
        else:
            groups.append(current_group)
            current_group = [word]
        last = word["Right"]
    groups.append(current_group)
    return [pd.DataFrame(group) for group in groups]

In [ ]:
x_clusters = cluster_x(header_words, header_words["Width"].min()*.95)

In [ ]:
len(x_clusters)

In [ ]:
left_bounds = pd.Series(
        cluster["Left"].min() for cluster in x_clusters[1:]
)
right_bounds = pd.Series(
    [cluster["Right"].max() for cluster in x_clusters[:-1]]
)
offsets = (right_bounds - left_bounds) / 2
full_left = pd.concat([pd.Series([0]), left_bounds + offsets]).reset_index(drop=True)
full_right = pd.concat([right_bounds - offsets, pd.Series([1])]).reset_index(drop=True)
col_spans = full_left.combine(full_right, lambda x, y: (x, y))

In [ ]:
col_spans

In [ ]:
full_left

In [ ]:
test_lines.loc[
    test_lines["Height"] > 5*test_lines["Width"],
    "Page"
].value_counts()

In [ ]:
def id_rotated_pages(df):
    return df.loc[
        df["Height"] > 5*df["Width"],
        "Page"
    ].value_counts().index.values


def rotate_pages(df):
    rotated_pages = id_rotated_pages(
        df.loc[
            df["BlockType"] == "LINE"
        ]
    )
    return df.loc[
        df["Page"].isin(rotated_pages)
    ].apply(rotate, axis=1)


def rotate(textract_obj):
    height = textract_obj["Height"]
    width = textract_obj["Width"]
    left = textract_obj["Left"]
    right = textract_obj["Right"]
    top = textract_obj["Top"]
    bottom = textract_obj["Bottom"]
    midpoint_x = textract_obj["Midpoint_X"]
    midpoint_y = textract_obj["Midpoint_Y"]
    new_obj = textract_obj.copy()
    new_obj["Height"] = width
    new_obj["Width"] = height
    new_obj["Left"] = bottom
    new_obj["Right"] = top
    new_obj["Top"] = left
    new_obj["Bottom"] = right
    new_obj["Midpoint_X"] = midpoint_y
    new_obj["Midpoint_Y"] = midpoint_x
    return new_obj

In [ ]:
rotate(test_words.loc[test_words["Page"] == 26].iloc[0])

In [ ]:
test_words.loc[test_words["Page"] == 26].iloc[0]

In [ ]:
id_rotated_pages(test_lines)

In [ ]:
table_test_df.loc[
    table_test_df["Page"].isin(id_rotated_pages(test_lines))
].apply(rotate, axis=1)

In [ ]:
rotate_pages(table_test_df)

In [ ]:
pd.cut(header_words["Left"], 6)

In [ ]:
pd.cut(header_words["Right"], 6).unique()[0]

In [ ]:
header_words.loc[header_words["Left"] < 0.0872]

In [ ]:
test_extractor.fields.count()

In [ ]:
.95*1.05